# Assignment 4 for Course 1MS041
Make         sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline         and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `mammography.mat` that you can download from [http://odds.cs.stonybrook.edu/mammography-dataset/](http://odds.cs.stonybrook.edu/mammography-dataset/). Below you can find the code to load this file into `X` and `Y`, you just need to put the file in your `data` folder. During mammography the doctor would be looking for something called `calcification`, which is calcium deposits in the breast tissue and is used as an indicator of cancer. In this dataset the `X` corresponds to some measurements, there are 6 features. The `Y` is a 0-1 label where 1 represents calcification and 0 does not.

1. [3p] Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, I have prepared a shuffling with a fixed seed, this way I can make sure that we all did the same splits. That is [train,test,validation] is the splitting layout.
2. [4p] Train a machine learning model on the training data (you are free to choose it yourself). Hint: you could always try `LogisticRegression`, but for it to work well you would need `StandardScaler` and put everything in a `Pipeline`.
3. [3p] Use the classification report from `Utils` and compute the intervals for precision-recall etc on the test set with `union_bound correction` with 95% confidence.
4. [3p] You are interested in minimizing the average cost of your classifier, but first we must define it:
    * If someone is calcified but classified as not, we say it costs 30 **(this is the worst scenario)** 
    * If someone is not calcified but classified as calcified, we say it costs 5 **(this probably only costs extra investigation)**
    * If someone is calcified but classified as calcified, costs 0 **(We did the right thing, no cost)**
    * If someone is not calcified but classified as not, costs 0 **(We did the right thing, no cost)**.

complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). What would be the cost of having a model that always classifies someone as not calcified on the test set?

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, we do that by checking say 100 evenly spaced proposal thresholds between 0 and 1, and use our testing data to compute the cost.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 95% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted, if we define the random variable
$$
    C = 30(1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
$$
then $C$ denotes the cost of a randomly chosen patient. In the above we are estimating $\mathbb{E}[C]$ using the empirical mean. However, since the number of calcifications in the population is fairly small and our classifier probably has a fairly high precision for the $0$ class, then $C$ should have fairly small variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [16]:
import scipy.io as so
import numpy as np
data = so.loadmat('data/mammography.mat')

np.random.seed(0)
shuffle_index = np.arange(0,len(data['X']))
np.random.shuffle(shuffle_index)

X = data['X'][shuffle_index,:]
Y = data['y'][shuffle_index,:].flatten()
# print(data)

In [17]:
import Utils
import math
n = len(X)
# print(n)
# Part 1

# Split the X,Y into three parts, make sure that the sizes are
# (6709, 6), (1677, 6), (2797, 6), (6709,), (1677,), (2797,)

# X_train, X_test, X_valid, Y_train, Y_test, Y_valid = X[:math.floor(0.6*n)], X[math.floor(0.6*n) : math.floor(0.75*n)-1], X[math.floor(0.75*n)-1:], Y[:math.floor(0.6*n)], Y[math.floor(0.6*n) : math.floor(0.75*n)-1], Y[math.floor(0.75*n)-1:]
X_train, X_test, X_valid, Y_train, Y_test, Y_valid = Utils.train_test_validation(X, Y, test_size=0.15, validation_size=0.25, shuffle=False)
# print(X_train.shape)
# print(X_test.shape)
# print(X_valid.shape)

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
# Part 2
# Use X_train,Y_train to train a model from sklearn. Make sure it has the predict_proba function
# for instance LogisticRegression

model = Pipeline([
    ('scale', StandardScaler()),
    ('clf', LogisticRegression())
])
model.fit(X_train, Y_train)

Pipeline(steps=[('scale', StandardScaler()), ('clf', LogisticRegression())])

In [19]:

# Part 3

# Compute precision and recall on the test set using 
from Utils import classification_report_interval

# Each precision and recall should be a tuple, for instance you can write
# precision0 = (0.9,0.95)
Y_predict = model.predict(X_test)
report = classification_report_interval(y_true=Y_test, y_pred=Y_predict, alpha=0.05)
# print(report)
precision0 = (0.95,1.00)
recall0 = (0.96,1.00)
precision1 = (0.54,1.00)
recall1 = (0.23,0.73)

# The code below will check that you supply the proper type
assert(type(precision0) == tuple)
assert(len(precision0) == 2)
assert(type(recall0) == tuple)
assert(len(recall0) == 2)
assert(type(precision1) == tuple)
assert(len(precision1) == 2)
assert(type(recall1) == tuple)
assert(len(recall1) == 2)

In [22]:
import Utils
# Part 4

def cost(model,threshold,X,Y):
    pred_proba = model.predict_proba(X)[:,1]
    predictions = (pred_proba >= threshold)*1
    costs = [30*(1-predictions[i])*Y[i] + 5*(1-Y[i])*predictions[i] for i in range(len(predictions))]
    # interval = compute_confidence_interval_bounded(costs, 0.05, min(costs), max(costs))
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost (cost per person)
    
    return sum(costs)/len(costs)

def interval(model,threshold,X,Y):
    pred_proba = model.predict_proba(X)[:,1]
    predictions = (pred_proba >= threshold)*1
    costs = [30*(1-predictions[i])*Y[i] + 5*(1-Y[i])*predictions[i] for i in range(len(predictions))]
    interval = Utils.compute_confidence_interval_bounded(costs, 0.05, min(costs), max(costs))
    
    return interval

In [24]:
# import matplotlib.pyplot as plt
# from sklearn import metrics
# 
# y_pred_proba = model.predict_proba(X_test)[:,1]
# fpr, tpr, _ = metrics.roc_curve(Y_test, # y_pred_proba)
# plt.plot(fpr,tpr)
# plt.ylabel('True Positive Rate')
# plt.xlabel('False Positive Rate')
# plt.show()
# Part 4

# Fill in the naive cost of the model that would classify all as non-calcified on the test set
naive_Y = [0 for i in range(len(Y))]

naive_cost = cost(model, 1, X, Y) #, naive_interval 
print(naive_cost)

0.6974872574443352


In [25]:
splits = np.linspace(0, 1, 100)
# print(splits)
cost_tests = []
for s in splits: 
    cost_test = cost(model, round(s, 8), X_test, Y_test)
    cost_tests.append((round(s, 8), cost_test))

costs = [cost_tests[i][1] for i in range(len(cost_tests))]
# Part 5
for val in cost_tests:
    if val[1] == min(costs):
        optimal_threshold = val[0]
        cost_at_optimal_threshold = val[1]
print(optimal_threshold , cost_at_optimal_threshold)

0.11111111 0.3518187239117472


In [26]:
#cost = 0.291383625
# Part 6

cost_at_optimal_threshold_validation, interval_validation = cost(model, optimal_threshold, X_valid, Y_valid), interval(model, optimal_threshold, X_valid, Y_valid)
# print(cost_at_optimal_threshold_validation)
# Report the cost interval as a tuple cost_interval = (a,b)
cost_interval = interval_validation
# print(cost_interval)
print(cost_at_optimal_threshold_validation, interval_validation , cost_interval)
# The code below will tell you if you filled in the intervals correctly
assert(type(cost_interval) == tuple)
assert(len(cost_interval) == 2)

AttributeError: module 'Utils' has no attribute 'compute_confidence_interval_bounded'

In [106]:

# Part 7
pred_proba_validation = model.predict_proba(X_valid)[:,1]
predictions = (pred_proba_validation >= optimal_threshold)*1
costs = [30*(1-predictions[i])*Y_valid[i] + 5*(1-Y_valid[i])*predictions[i] for i in range(len(predictions))]
temp = [(c - (sum(costs)/len(predictions)))**2 for c in costs]

variance_of_C = sum(temp)/len(temp)
# print(variance_of_C)
# 7.672650698709501

In [107]:
from Utils import bennett_epsilon
# Part 7

# A 95% confidence interval of the random variable C using Bennett's inequality
epsilon = bennett_epsilon(len(costs), max(costs), variance_of_C**0.5, 0.05)

interval_of_C = (cost_at_optimal_threshold_validation-epsilon, cost_at_optimal_threshold_validation+epsilon)
# print(interval_of_C)

assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)

Numerical error -2.3245294578089215e-16
